In [1]:
import requests
import pandas as pd
import json
import plotly.graph_objects as go

# Obatining cattle data for the analysis, source IBGE(in English, "Brazilian Institute of Geography and Statistics").

In [22]:
url = (
    "https://apisidra.ibge.gov.br/values/"
    "t/3939/"
    "n1/all/"
    "n2/all/"
    "n8/all/"
    "v/all/"
    "p/all/"
    "c79/2670"
)
cattle_raw = pd.DataFrame(json.loads(requests.get(url).text)[1:])
# converting numeric types
cattle_raw[["NC", "MC", "D1C", "D2C", "D4C"]] = cattle_raw[["NC", "MC", "D1C", "D2C", "D4C"]].astype(int)
cattle_raw["V"] = cattle_raw["V"].apply(pd.to_numeric, errors='coerce')
# converting datetime types
cattle_raw[["D3C", "D3N"]] = cattle_raw[["D3C", "D3N"]].apply(pd.to_datetime)
# setting index as datetime column
cattle_raw.index = cattle_raw["D3C"]
cattle_raw.dropna(inplace=True)
# cattle_raw.to_csv("cattle_raw_data.csv")

# Obtaining Amazon deforestation rates for analysis, source PRODES (in English, "Project for Monitoring Deforestation in the Amazon by Satellites")

In [17]:
def_raw = pd.read_html("http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes")[1]
def_raw.columns = def_raw.iloc[0, :]
def_raw = def_raw.iloc[ 1:-2 , : ].copy()
def_raw["Ano/Estados"] = pd.to_datetime(def_raw["Ano/Estados"])
def_raw.set_index("Ano/Estados", inplace=True)
def_raw.rename_axis("date", inplace=True)
def_raw = def_raw.apply(pd.to_numeric)
# def_raw.to_csv("amazon_def.csv", index=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 34 entries, 1988-01-01 to 2021-01-01
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   AC         34 non-null     int64
 1   AM         34 non-null     int64
 2   AP         34 non-null     int64
 3   MA         34 non-null     int64
 4   MT         34 non-null     int64
 5   PA         34 non-null     int64
 6   RO         34 non-null     int64
 7   RR         34 non-null     int64
 8   TO         34 non-null     int64
 9   AMZ LEGAL  34 non-null     int64
dtypes: int64(10)
memory usage: 2.9 KB


In [4]:
cattle_raw.query("NN == 'Mesorregião Geográfica'")

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
D3C,,,,,,,,,,,,,
1974-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1974-01-01,1974-01-01,2670,Bovino
1975-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1975-01-01,1975-01-01,2670,Bovino
1976-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1976-01-01,1976-01-01,2670,Bovino
1977-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1977-01-01,1977-01-01,2670,Bovino
1978-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1978-01-01,1978-01-01,2670,Bovino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-01,8,Mesorregião Geográfica,24,Cabeças,93141.0,5301,Distrito Federal - DF,105,Efetivo dos rebanhos,2017-01-01,2017-01-01,2670,Bovino
2018-01-01,8,Mesorregião Geográfica,24,Cabeças,90199.0,5301,Distrito Federal - DF,105,Efetivo dos rebanhos,2018-01-01,2018-01-01,2670,Bovino
2019-01-01,8,Mesorregião Geográfica,24,Cabeças,84425.0,5301,Distrito Federal - DF,105,Efetivo dos rebanhos,2019-01-01,2019-01-01,2670,Bovino


In [5]:
# list of amazon states (abbreviations)
amazon = ["AC", "AM", "AP", "MA", "MT", "PA", "RO", "RR", "TO"]

In [6]:
# querying from raw data only mesoregions within amazon states
amazon_df = cattle_raw[cattle_raw['D1N'].apply(lambda x: any(elem in x for elem in amazon))].copy()
amazon_df

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
D3C,,,,,,,,,,,,,
1974-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1974-01-01,1974-01-01,2670,Bovino
1975-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1975-01-01,1975-01-01,2670,Bovino
1976-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1976-01-01,1976-01-01,2670,Bovino
1977-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1977-01-01,1977-01-01,2670,Bovino
1978-01-01,8,Mesorregião Geográfica,24,Cabeças,NaN,1101,Madeira-Guaporé - RO,105,Efetivo dos rebanhos,1978-01-01,1978-01-01,2670,Bovino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-01,8,Mesorregião Geográfica,24,Cabeças,2987083.0,5105,Sudeste Mato-grossense - MT,105,Efetivo dos rebanhos,2017-01-01,2017-01-01,2670,Bovino
2018-01-01,8,Mesorregião Geográfica,24,Cabeças,2971248.0,5105,Sudeste Mato-grossense - MT,105,Efetivo dos rebanhos,2018-01-01,2018-01-01,2670,Bovino
2019-01-01,8,Mesorregião Geográfica,24,Cabeças,3079127.0,5105,Sudeste Mato-grossense - MT,105,Efetivo dos rebanhos,2019-01-01,2019-01-01,2670,Bovino


In [29]:
# creating function to obaint IBGE mesoregion data
def download_geojson(mesoregion_id):
    url = f"https://servicodados.ibge.gov.br/api/v3/malhas/mesorregioes/{mesoregion_id}?formato=application/vnd.geo+json"
    response = requests.get(url)
    with open(f"geojson\{mesoregion_id}.json", "w") as f:
        f.write(response.text)

In [30]:
# Obtaining geo borders from each mesoregion trough IBGE (Brazilian Institute of Geography and Statistics) api.
for meso_id in amazon_df["D1C"].unique():
    download_geojson(meso_id)

In [100]:
# Creating map figure
with open(f'geojson/1101.json', 'r') as f:
    geojson_data = json.load(f)["features"][0]["geometry"]["coordinates"][0]
trace = go.Scattermapbox(
    lon=[coord[0] for coord in geojson_data],
    lat=[coord[1] for coord in geojson_data],
    mode='lines',
    line=dict(width=1, color='blue'),
    fill='toself',
    text = f"Rondonia <br> lalala",
    hoverinfo= "text"
)

fig = go.Figure(trace)

fig.update_layout(
    mapbox=dict(
        style='carto-positron',
        zoom=3,
        center=dict(lon=-55, lat=-15)
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=600,
    width=800
)

fig.show()